# Instructions

You should run this notebook cell-by-cell as user-interaction might be required and several plots are created displaying the cell placement.

For more information about the science in the use case please consult the [Guidebook](https://collab.humanbrainproject.eu/#/collab/1655/nav/18580)

----

The following cell imports all required python modules.

In [ ]:
import os
import sys
import time
import datetime
import logging
# Configure logger to see warnings from the underlying libraries
logging.basicConfig(stream=sys.stderr)
import requests
import requests.packages.urllib3 as urllib3
urllib3.disable_warnings()
from voxcell.core import CellCollection, VoxelData, RegionMap
from voxcell import build, math
from voxcellview.widget import VoxcellWidget
from brainbuilder.utils import bbp
import numpy as np
from brainbuilder.select_region import select_hemisphere

cells = CellCollection()

# Configuration

This notebook is configured to use the **CA1** region for a rat. The next cell will copy input parameters from hippocampus collab.


In [ ]:
SPECIE = 'rat' 
ROI = 'CA1'
!curl https://bbp.epfl.ch/public/hippocampus/recipe/recipe.tar | tar x
DATA_DIR = "/home/jovyan"
BUILDER_RECIPE = os.path.join(DATA_DIR, SPECIE + '/' + ROI + "/bionames/builderRecipeAllPathways.xml")
V4NEURONDB = os.path.join(DATA_DIR, SPECIE + '/' + ROI + "/bionames/v4neuronDB.dat")

# Circuit Building pipeline

### Select Region of interest

Define the region of interest and retrieve the volumetric information from the Neuroinformatics Platform.
The region of interest, in this case, is the hippocampus.  The region of interest defines each of the voxels that compose the hippocampus, and tags their value as to which layer they are part of.  These come from atlas registrations that have been imported by Neuroinformatics Platform, in our case we use the Allen Brain institute atlas which has regions tagged per voxel at 25um.

In [ ]:
NIP_URL = "https://nip.humanbrainproject.eu/api/analytics"
atlas_id = '9B1F97DD-13B8-4FCF-B9B1-59E4EBE4B5D8'
region_id = 200
subregions =  ('SLM', 'SR', 'SP', 'SO')

### Retrieve region structure and volumetric data

In [ ]:
hierarchy_url = "{}/atlas/releases/{}/filters/brain_region/{}".format(NIP_URL, atlas_id, region_id)
hierarchy_data = requests.get(hierarchy_url)
region_map = RegionMap.from_dict(hierarchy_data.json())
download_url = "{}/atlas/download?uri={}/brain_regions/brain_region/{}.nrrd".format(NIP_URL, atlas_id, region_id)
nrrd_file = "{}_{}.nrrd".format(atlas_id, region_id)
with open(nrrd_file, 'wb') as f:
    data = requests.get(download_url)
    for chunk in data:
        f.write(chunk)

### Download the volumetric data locally

In [ ]:
region_ids = [id_ for region_name in subregions for id_ in region_map.ids(region_name)]

annotation = VoxelData.load_nrrd(nrrd_file)

## Compute cell densities

In [ ]:
region_layers_map = {}
for region_name in subregions:
    for rids in region_map.sub(region_id).ids(region_name):
        region_layers_map[rids] = (region_name, )


### Compute density based on a custom recipe

In [ ]:
density = bbp.load_recipe_density(BUILDER_RECIPE, annotation, region_layers_map)

## Positions

Using the 'Builder Recipe', which has been curated by hand to account for the densities of cells in different layers, and their types, the soma positions are randomly distributed based on these densities.  Each run of the cell will result in a new hippocampus, as the positions will be different.

In [ ]:
from brainbuilder.cell_positioning import cell_positioning
total_cell_count = 3500 # Bezaire and Soltesz, 2013; West et al 1991
print "WARNING: total number of cells scaled by 1/100"
print 'total cell', total_cell_count, 'selected for region', ROI

cells.positions = cell_positioning(density, total_cell_count)

### Display positions and show as dataframe

In [ ]:
VoxcellWidget().show_points('position', cells.as_dataframe())

In [ ]:
cells.as_dataframe()

## Build.EI: Define Excitatory and Inhibitory properties

Again, using the 'Builder Recipe', each of the cells that have been given a position are then given a label describing whether they are excitatory or inhibitory cells.  These ratios (ie: probabilites) could vary per region.

### Input parameters

In [ ]:
from brainbuilder.assignment_synapse_class import assign_synapse_class_from_spatial_dist
recipe_data = bbp.get_distribution_from_recipe(BUILDER_RECIPE)
recipe_sdist = bbp.transform_recipe_into_spatial_distribution(annotation, recipe_data, region_layers_map)

### Run module

In [ ]:
chosen_synapse_class = assign_synapse_class_from_spatial_dist(cells.positions, recipe_sdist)
cells.add_properties(chosen_synapse_class)

### Display EI positions

In [ ]:
VoxcellWidget().show_property('synapse_class', cells.as_dataframe())

## Build.Composition.ME: Define morphological and electrical types

In tandem with selecting the label for excitatory or inhibitory-ness, the morphology-types of the cell are chosen, again based on the probability in the region as defined by the 'Builder Recipe'.  This does not mean an actual morphology is chosen, yet, just that the cell will only have the actual morphology assigned from a pool based on the me-type.

### Assign Morphological and Electrical types based on a custom recipe

In [ ]:
from brainbuilder.assignment_metype import assign_metype
chosen_me = assign_metype(cells.positions, cells.properties.synapse_class, recipe_sdist)
cells.add_properties(chosen_me)

### Display mtype property

In [ ]:
VoxcellWidget().show_property('mtype', cells.as_dataframe())

### Display etype property

In [ ]:
VoxcellWidget().show_property('etype', cells.as_dataframe())

## Build.Placement: Morphology assignment

Once the ME-type has been chosen, we can pick from a pool of morphologies that satisify this type, and randomly select one that fits.  By doing it this way, each run will produce a different usage of the morphologies that are available.  In addition, as more more morphologies become available, the previous steps may not need to be re-run: only this step.

In [ ]:
from brainbuilder.assignment_morphology import assign_morphology
from scipy.ndimage import distance_transform_edt

### Input parameters

In [ ]:
neurondb = bbp.load_neurondb_v4(V4NEURONDB)
hippo_mask = build.mask_by_region_ids(annotation.raw, region_ids)
distance_to_pia = distance_transform_edt(hippo_mask)
neuron_sdist = bbp.transform_neurondb_into_spatial_distribution(annotation,
                                                                 neurondb,
                                                                 region_layers_map,
                                                                 distance_to_pia,
                                                                 percentile=0.92)

### Assign morphologies

In [ ]:
chosen_morphology = assign_morphology(cells.positions, cells.properties[['mtype', 'etype']], neuron_sdist)
cells.add_properties(chosen_morphology)

### Display assigned morphologies

In [ ]:
VoxcellWidget().show_property('morphology', cells.as_dataframe())

## Orientation assignment

It is necessary to calculate how the morphology will be oriented in the space.
A series of vector fields are calculated for this pourpose.

In the present model, the correct positioning of the morphologies require
three vector fields which are parallel to the three main axes of the hippocampus:
septo-temporal axis, transversal axis, and radial axis.

### Compute orientation vector fields

In [ ]:
first_region_ids = region_map.ids('SO')
last_region_ids = region_map.ids('SLM')
ROI_ids = region_map.ids('CA1') | region_map.ids('CA2') | region_map.ids('CA3')

print 'first_region_ids =', first_region_ids
print 'last_region_ids =', last_region_ids
print 'ROI_ids =', ROI_ids

In [ ]:
from brainbuilder.orientation_field_hippo import compute_orientation_field
orientation_field = compute_orientation_field(annotation, ROI_ids, first_region_ids, last_region_ids)
from voxcell import vector_fields as vf
sub_fields = vf.split_orientation_field(orientation_field.raw)

### Display 'Z' direction vector field

In [ ]:
dimensions = orientation_field.voxel_dimensions.astype('int64')
VoxcellWidget().show_vectors('Z', sub_fields[2], 5000, dimensions)

### Display 'Y' direction vector field

In [ ]:
VoxcellWidget().show_vectors('Y', sub_fields[1], 5000, dimensions)

### Display 'X' direction vector field

In [ ]:
VoxcellWidget().show_vectors('X', sub_fields[0], 5000, dimensions)

### Assign orientations and save Circuit Placement file

In [ ]:
from brainbuilder.assignment_orientation import assign_orientations
cells.orientations = assign_orientations(cells.positions, orientation_field)
cells.remove_unassigned_cells()
time = datetime.datetime.now().strftime("%Y%m%dT%H%M%S")
# cells_filename = 'circuit_' + time + '.mvd3'
cells_filename = 'circuit.mvd3'
cells_path = DATA_DIR + 'mvd3/' + SPECIE + '/' + ROI + '/' + cells_filename

import time
timestamp = time.strftime('%Y%m%d_%H%M%S')
cells_filename = 'circuit_'+timestamp+'.mvd3'
cells_file = os.path.join(DATA_DIR, cells_filename)
cells.save(cells_file)

clients = get_hbp_service_client()
collab_path = get_collab_storage_path()

_=clients.storage.upload_file(cells_file, os.path.join(collab_path, cells_filename), 'text/plain')
print("The placement has been saved in the storage of this Collab under: '"+cells_filename+"'")